# Sports Analytics - Premier League <br>
Author: **Alvin Ong** <br>

------------------------------------------------------------------------

## Table of Contents
1. [Project Overview](#project-overview)
2. [Data Wrangling](#data-wrangling)
3. [Exploratory Data Analysis](#exploratory-data-analysis)
    - [Top Goal Scorers](#top-goal-scorers)
    - [Possession vs Goals](#possession-vs-goals)
    - [Player Trends Over Seasons](#player-trends-over-seasons)
    - [Player Clustering](#player-clustering)
4. [Conclusion & Next Steps](#conclusion-&-next-steps)

------------------------------------------------------------------------

## Project Overview

### Business Context
The Premier League is one of the most prestigious football leagues globally, attracting millions of viewers and generating significant financial and cultural impact. Football clubs, analysts, and enthusiasts are increasingly leveraging data to gain competitive advantages on and off the pitch.

This project originates from a sports analytics perspective, where I aim to derive meaningful insights from historical Premier League data. The data covers the last 6 seasons and includes the standard player-level and team-level performance metrics.

### Business Goals
The primary goal of this project is to explore, understand, and model player and team performances using statistical methods and machine learning techniques. The insights generated can help stakeholders answer key questions such as:
- Which players have been the most effective goal scorers and creators?
- How does a team's possession correlate with their goal-scoring success?
- Are there notable trends in player or team performance over time?
- Can we segment players into meaningful clusters based on their attributes?
- Are there correlations between defensive solidity and attacking output.
- Can we predict goal output using regression models.

### Expected Deliverables
The deliverable from this project is a comprehensive analysis notebook that will:
- Summarize key findings in the form of visualizations and insights.
- Identify patterns and trends for reporting purposes.
- Serve as a foundation for further advanced modeling (e.g., regression or forecasting).
- Be shared internally with analysts, coaches, or decision-makers in the form of a report or dashboard.

### Prior Knowledge and Data Context
The data is sourced from the FBref database via ```soccerdata``` and focuses exclusively on Premier League matches across the last 6 seasons. It includes detailed metrics such as goals, assists, possession, expected goals (xG), and progressive actions (actions that move the ball towards the opponent's goal line).

Note that this dataset does not currently include injury data or certain defensive metrics, which may limit the scope of some analyses. Additionally, xG values are based on modeling estimates and may not fully capture all nuances of match outcomes.

------------------------------------------------------------------------

## Data Wrangling

### Dataset Overview
In this analysis, we will be working with player and team statistics from the Premier League, loaded from a SQLite database. The primary tables include:

- **playerstatistics**: Contains player-level statistics such as goals, assists, expected goals (xG), expected assists (xA), progressive passes, and Yellow/Red Cards.
- **teamstatistics**: Contains team-level aggregate statistics like total goals, possession percentage, Yellow/Red cards, and squad usage metrics.

### Notes on Data Cleaning
The dataset has already been pre-processed and cleaned in a separate notebook prior to this analysis. The data cleaning process included:
- Flattened Table Headers
- Drop unneccesary columns
- Renamed Headers for better clarity
- Correcting inconsistent column data types.

All major wrangling tasks have been documented and executed previously to ensure this notebook focuses purely on analysis.

### Data Relationships
- **PlayerStatistics** is linked to **players** via `Player`.
- **PlayerStatistics** and **TeamStatstics** are linked to **Teams** via `Team`,  **Seasons** via `Name`, and **League** via `Name`.

The following section will load the cleaned dataset from the SQLite database and perform basic checks to verify its readiness for analysis.

### Data Loading

In [1]:
# importing dependencies
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [37]:
# Connect to SQLite database
conn = sqlite3.connect("../data/FBref_football_stats.db")

## Exploratory Data Analysis

### League-Level Analysis
- Overall season metrics (e.g., avg goals per game, xG trends, discipline trend)


In [71]:
# Overall Season metrics

query = """
SELECT 
    Season,
    AVG(Goals) AS AvgGoalsPerGame,
    AVG(ExpectedGoals) AS AvgExpectedGoals,
    AVG(YellowCards) AS AvgYellowCards,
    AVG(RedCards) AS AvgRedCards
FROM 
    TeamStatistics
WHERE
    Season != '2425'
GROUP BY 
    Season
ORDER BY 
    Season;
"""
season_metrics = pd.read_sql(query, conn)

# Plot trends for goals, and xG
fig1 = px.line(season_metrics, x='Season', y=[
            'AvgGoalsPerGame', 
            'AvgExpectedGoals'
            ],
            title="Overall Season Metrics: Goals and xG Trends",
            labels={"value": "Average Metric", "Season": "Season"},
            markers=True)
fig1.update_layout(legend_title_text="Metrics")
fig1.show()

# Plot trends for yellow and red cards
fig2 = px.line(season_metrics, x='Season', y=[
            'AvgYellowCards', 
            'AvgRedCards'
            ],
            title="Overall Season Metrics: Yellow and Red Cards Trends",
            labels={"value": "Average Metric", "Season": "Season"},
            markers=True)
fig2.update_layout(legend_title_text="Metrics")
fig2.show()

#### Insights: Total Goals vs Total xG (2019-2024)
**Positive Trend**:  
Both **average goals per game** and **average expected goals (xG)** have shown a positive trend for the last 4 seasons, indicating an overall improvement in attacking play and chance creation in the league.

**Notable Trends**:  
A significant gap is observed where average expected goals (xG) were **higher** than average goals scored, suggesting inefficiency in finishing or strong defensive performances during the 2019-2020 and 2022-2023 Seasons.

**General Observation**:  
The alignment between goals and xG in most seasons suggests that teams are generally converting chances at an expected rate.  
However, the notable gaps in the 1920 and 2223 seasons highlight potential areas for further analysis, such as whether these trends were driven by specific teams, players, or tactical shifts in the league.

#### Insights: Yellow and Red Card Trends
**Red Card Trends**:  
The **average red cards per game** have remained steady, hovering in the **1-2 range** across all seasons. This indicates consistent disciplinary patterns in terms of severe fouls or misconduct.

**Yellow Card Trends**:  
There has been a **steady increase in average yellow cards per game** over the past 4 seasons, suggesting a rise in less severe fouls or tactical fouls being penalized by referees.  
This trend may also reflect **stricter rules and refereeing standards**, leading to more frequent bookings for minor infractions.

**General Observation**:  
The stability in red cards suggests that teams and players are not engaging in significantly more dangerous play over time.  
The increase in yellow cards could indicate a combination of **stricter enforcement of rules, more aggressive defensive tactics**, or a rise in **tactical fouling** to disrupt opposition play. This trend warrants further analysis to understand its root causes.

### Team-Level Analysis
- Track possession percentages across seasons for each team
- Analyze team age profiles and their correlation with performance
- Compare progressive actions (ProgressiveCarries, ProgressivePasses) by team across seasons
- Study how teams' expected goals vs. actual goals have changed over time

In [72]:
# Track Possession Percentages Across Seasons for Each Team
query = """
SELECT 
    Team, 
    Season, 
    Possession AS AvgPossession
FROM 
    TeamStatistics
GROUP BY 
    Team, Season
ORDER BY 
    Team, Season;
"""
team_possession = pd.read_sql(query, conn)

fig_possession = px.line(team_possession, x='Season', y='AvgPossession', color='Team', markers=True,
                         title="Team Possession Percentages Across Seasons",
                         labels={"AvgPossession": "Average Possession (%)", "Season": "Season"},
                         hover_name='Team')

# Add a callback to highlight the selected team
fig_possession.update_layout(
    legend=dict(
        itemclick="toggleothers",
        itemdoubleclick="toggle"
    )
)

fig_possession.show()

In [ ]:
# Analyze Team Age Profiles and Their Correlation with Performance
query = """
SELECT 
    Team, 
    AVG(AverageAge) AS AvgAge, 
    SUM(GoalsPlusAssists) AS PerformanceMetric1,
    AVG(Possession) AS PerformanceMetric2
FROM 
    TeamStatistics
GROUP BY 
    Team
ORDER BY 
    Team;
"""
team_age_performance = pd.read_sql(query, conn)

# Create the interactive scatter plot for GoalsPlusAssists
fig_age_performance1 = px.scatter(team_age_performance, x='AvgAge', y='PerformanceMetric1', color='Team', size='PerformanceMetric1', hover_name='Team',
                                  title="Team Age Profiles and Their Correlation with GoalsPlusAssists",
                                  labels={"AvgAge": "Average Age", "PerformanceMetric1": "Goals Plus Assists"})
fig_age_performance1.update_layout(showlegend=True)
fig_age_performance1.show()

# Create the interactive scatter plot for Possession
fig_age_performance2 = px.scatter(team_age_performance, x='AvgAge', y='PerformanceMetric2', color='Team', size='PerformanceMetric2', hover_name='Team',
                                  title="Team Age Profiles and Their Correlation with Possession",
                                  labels={"AvgAge": "Average Age", "PerformanceMetric2": "Possession (%)"})
fig_age_performance2.update_layout(showlegend=True)
fig_age_performance2.show()

### 1️⃣ Top goal scorers & most efficient
- Identify the top-performing players in terms of goal output.
- Compare actual goals with expected goals (xG) to measure efficiency.
- Analyze goal efficiency using Goals per 90 minutes metric
- Compare raw goals vs. non-penalty goals to identify penalty specialists
- Highlight players who consistently outperform their xG across multiple seasons.

In [51]:
# top scorer across last 6 seasons
query = """
    SELECT p.Name, ps.Season, ps.Team, ps.Goals, ps.ExpectedGoals 
    FROM PlayerStatistics ps
    JOIN Players p ON ps.Player = p.Name
    ORDER BY ps.Goals DESC
    LIMIT 10;
"""
top_scorers = pd.read_sql(query, conn)
print(top_scorers)

                        Name Season             Team  Goals  ExpectedGoals
0             Erling Haaland   2223  Manchester City     36           28.4
1                 Harry Kane   2223        Tottenham     30           21.4
2             Erling Haaland   2324  Manchester City     27           29.2
3              Mohamed Salah   2425        Liverpool     27           22.0
4                Jamie Vardy   1920   Leicester City     23           19.2
5                 Harry Kane   2021        Tottenham     23           20.1
6              Mohamed Salah   2122        Liverpool     23           22.7
7              Son Heung-min   2122        Tottenham     23           15.9
8  Pierre-Emerick Aubameyang   1920          Arsenal     22           14.7
9                 Danny Ings   1920      Southampton     22           16.2


In [52]:
# Create the interactive scatter plot
fig = px.scatter(top_scorers, x="ExpectedGoals", y="Goals", color="Name", hover_name="Name",
                 title="Top Scorers across last 6 seasons", hover_data=["Season"])

# Add a diagonal line
fig.add_shape(type="line",
              x0=0, y0=0,
              x1=top_scorers['ExpectedGoals'].max(), y1=top_scorers['ExpectedGoals'].max(),
              line=dict(color="Black", dash="dash"))

fig.update_layout(showlegend=True)
fig.show()

In [53]:
# Execute the SQL query
query = """
SELECT 
    p.Name AS Player, p.Position AS Position, ps.Team AS Team,
    SUM(ps.Goals) AS RawGoals, 
    SUM(ps.NonPenaltyGoals) AS NonPenaltyGoals
FROM 
    PlayerStatistics ps
JOIN 
    Players p ON ps.Player = p.Name
GROUP BY 
    p.Name
ORDER BY 
    RawGoals DESC;
"""
df = pd.read_sql(query, conn)

# Create the interactive scatter plot
fig = px.scatter(df, x="RawGoals", y="NonPenaltyGoals", color="Player", hover_name="Player",
                 title="Raw Goals vs. Non-Penalty Goals", hover_data=["Position", "Team"])

# Add a diagonal line
fig.add_shape(type="line",
              x0=0, y0=0,
              x1=df['RawGoals'].max(), y1=df['NonPenaltyGoals'].max(),
              line=dict(color="Black", dash="dash"))

fig.update_layout(showlegend=True)
fig.show()

In [54]:
# Execute the SQL query
query = """
SELECT 
    p.Name AS Player, 
    SUM(ps.Goals) AS TotalGoals, 
    SUM(ps.ExpectedGoals) AS TotalExpectedGoals,
    SUM(ps.MinutesPlayed) AS TotalMinutes,
    (SUM(ps.Goals) / (SUM(ps.MinutesPlayed) / 90.0)) AS GoalsPer90,
    (SUM(ps.ExpectedGoals) / (SUM(ps.MinutesPlayed) / 90.0)) AS ExpectedGoalsPer90
FROM 
    PlayerStatistics ps
JOIN 
    Players p ON ps.Player = p.Name
GROUP BY 
    p.Name
HAVING 
    SUM(ps.MinutesPlayed) >= 450 AND SUM(ps.Goals) >= 10
ORDER BY 
    GoalsPer90 DESC;
"""
df = pd.read_sql(query, conn)

# Add a new column to categorize overperformers and underperformers
df['Performance'] = df.apply(lambda row: 'Overperformer' if row['GoalsPer90'] > row['ExpectedGoalsPer90'] else 'Underperformer', axis=1)

# Calculate the top 10% threshold for xG/90
top_10_percent_threshold = df['ExpectedGoalsPer90'].quantile(0.9)

# Filter the DataFrame to include only the top 10% of players based on xG/90
top_10_percent_df = df[df['ExpectedGoalsPer90'] >= top_10_percent_threshold]

# Create the interactive bubble chart
fig = px.scatter(top_10_percent_df, x="ExpectedGoalsPer90", y="GoalsPer90", size="TotalMinutes", color="Performance", hover_name="Player",
                 title="Top 10% Players: G/90 vs xG/90 with Bubble Size Representing Minutes Played", 
                 labels={"ExpectedGoalsPer90": "xG/90", "GoalsPer90": "G/90"},
                 hover_data=["TotalGoals", "TotalExpectedGoals", "TotalMinutes"])

fig.update_layout(showlegend=True)
fig.show()

In [55]:
# Execute the SQL query
query = """
SELECT 
    p.Name AS Player, 
    SUM(ps.Goals) AS TotalGoals, 
    SUM(ps.ExpectedGoals) AS TotalExpectedGoals,
    (SUM(ps.Goals) - SUM(ps.ExpectedGoals)) AS GoalDifference
FROM 
    PlayerStatistics ps
JOIN 
    Players p ON ps.Player = p.Name
GROUP BY 
    p.Name
ORDER BY 
    GoalDifference DESC
LIMIT 10;
"""
df = pd.read_sql(query, conn)

# Create the interactive scatter plot
fig = px.scatter(df, x="TotalExpectedGoals", y="TotalGoals", color="Player", hover_name="Player",
                 title="Top 10 Clinical Finishers: Actual Goals vs Expected Goals", 
                 labels={"TotalExpectedGoals": "Expected Goals", "TotalGoals": "Actual Goals"},
                 hover_data=["GoalDifference"])

# Add a diagonal line
fig.add_shape(type="line",
              x0=0, y0=0,
              x1=df['TotalExpectedGoals'].max(), y1=df['TotalGoals'].max(),
              line=dict(color="Black", dash="dash"))

fig.update_layout(showlegend=True)
fig.show()

### 2️⃣ Team Performance Evolution
- Track possession percentages across seasons for each team
- Analyze team age profiles and their correlation with performance
- Compare progressive actions (ProgressiveCarries, ProgressivePasses) by team across seasons
- Study how teams' expected goals vs. actual goals have changed over time

In [15]:
# Compare Progressive Actions (ProgressiveCarries, ProgressivePasses) by Team Across Seasons
query = """
SELECT 
    Team, 
    Season, 
    SUM(ProgressiveCarries) AS ProgressiveCarries, 
    SUM(ProgressivePasses) AS ProgressivePasses
FROM 
    TeamStatistics
WHERE
    Season != 2425
GROUP BY 
    Team, Season
ORDER BY 
    Team, Season;
"""
team_progressive_actions = pd.read_sql(query, conn)

fig_carries = px.line(team_progressive_actions, x='Season', y='ProgressiveCarries', color='Team', markers=True,
                      title="Progressive Carries by Team Across Seasons",
                      labels={"ProgressiveCarries": "Progressive Carries", "Season": "Season"},
                      hover_name='Team')
fig_carries.update_layout(
    legend=dict(
        itemclick="toggleothers",
        itemdoubleclick="toggle"
    ))
fig_carries.show()

fig_passes = px.line(team_progressive_actions, x='Season', y='ProgressivePasses', color='Team', markers=True,
                     title="Progressive Passes by Team Across Seasons",
                     labels={"ProgressivePasses": "Progressive Passes", "Season": "Season"},
                     hover_name='Team')
fig_passes.update_layout(
    legend=dict(
        itemclick="toggleothers",
        itemdoubleclick="toggle"
    )
)
fig_passes.show()

In [ ]:
# Study How Teams' Expected Goals vs. Goals Have Changed Over Time
query = """
SELECT 
    Team, 
    Season, 
    SUM(ExpectedGoals) AS ExpectedGoals, 
    SUM(Goals) AS Goals
FROM 
    TeamStatistics
WHERE
    Season != 2425
GROUP BY 
    Team, Season
ORDER BY 
    Team, Season;
"""
team_goals = pd.read_sql(query, conn)

fig_expected_goals = px.line(team_goals, x='Season', y='ExpectedGoals', color='Team', markers=True,
                             title="Expected Goals by Team Across Seasons",
                             labels={"ExpectedGoals": "Expected Goals", "Season": "Season"},
                             hover_name='Team')
fig_expected_goals.update_layout(
    legend=dict(
        itemclick="toggleothers",
        itemdoubleclick="toggle"
    )
)
fig_expected_goals.show()

fig_actual_goals = px.line(team_goals, x='Season', y='Goals', color='Team', markers=True,
                           title="Goals by Team Across Seasons",
                           labels={"Goals": "Goals", "Season": "Season"},
                           hover_name='Team')
fig_actual_goals.update_layout(
    legend=dict(
        itemclick="toggleothers",
        itemdoubleclick="toggle"
    )
)
fig_actual_goals.show()

In [18]:
# Analyze team efficiency to convert possession into goals
query = """
SELECT Team, AVG(Possession) AS AvgPossession, SUM(Goals) AS TotalGoals
FROM TeamStatistics
GROUP BY Team
ORDER BY TotalGoals DESC;
"""
team_efficiency = pd.read_sql(query, conn)

# Create the interactive scatter plot
fig = px.scatter(team_efficiency, x='AvgPossession', y='TotalGoals', color='Team',
                 title="Team Efficiency: Possession vs Goals",
                 labels={"AvgPossession": "Average Possession (%)", "TotalGoals": "Total Goals"},
                 hover_name='Team')

# Fit a linear regression model
X = team_efficiency[['AvgPossession']]
y = team_efficiency['TotalGoals']
model = LinearRegression().fit(X, y)
line_x = np.linspace(X.min(), X.max(), 100)
line_y = model.predict(line_x)

# Add the regression line to the plot
fig.add_trace(go.Scatter(x=line_x.flatten(), y=line_y, mode='lines', name='Regression Line', line=dict(color='black')))

fig.update_layout(showlegend=True)
fig.show()

e:\Users\abin\Documents\Data Analysis Docs\Python Projects\sports-analytics-premier-league\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



### 3️⃣ Player trends
- Identify rising stars
- Spot consistent top performers across multiple seasons.

In [49]:
# Analyze player improvement across seasons
query = """
SELECT 
    ps.Player AS PlayerID, -- Include PlayerID
    p.Name AS Player,
    p.Age AS Age,
    ps.Season,
    SUM(ps.GoalsPlusAssists) AS TotalGoalsPlusAssists,
    SUM(ps.ProgressivePasses) AS TotalProgressivePasses,
    SUM(ps.ProgressiveCarries) AS TotalProgressiveCarries
FROM 
    PlayerStatistics ps
JOIN 
    Players p ON ps.Player = p.Name
GROUP BY 
    ps.Player, p.Name, ps.Season
ORDER BY 
    ps.Player, ps.Season;
"""
player_stats = pd.read_sql(query, conn)

# Calculate year-over-year improvement
player_stats['GoalsPlusAssistsImprovement'] = player_stats.groupby('PlayerID')['TotalGoalsPlusAssists'].diff()
player_stats['ProgressivePassesImprovement'] = player_stats.groupby('PlayerID')['TotalProgressivePasses'].diff()
player_stats['ProgressiveCarriesImprovement'] = player_stats.groupby('PlayerID')['TotalProgressiveCarries'].diff()

# Drop rows where improvements cannot be calculated (e.g., first season for each player)
player_stats = player_stats.dropna(subset=['GoalsPlusAssistsImprovement', 'ProgressivePassesImprovement', 'ProgressiveCarriesImprovement'])

# Normalize the improvements
improvement_columns = ['GoalsPlusAssistsImprovement', 'ProgressivePassesImprovement', 'ProgressiveCarriesImprovement']
for col in improvement_columns:
    mean = player_stats[col].mean()
    std = player_stats[col].std()
    player_stats[f'Normalized{col}'] = (player_stats[col] - mean) / std

# Calculate total normalized improvement
player_stats['TotalImprovement'] = (
    player_stats['NormalizedGoalsPlusAssistsImprovement'] +
    player_stats['NormalizedProgressivePassesImprovement'] +
    player_stats['NormalizedProgressiveCarriesImprovement']
)

# Aggregate total improvement by player
rising_stars = player_stats.groupby(['PlayerID', 'Player', 'Age'], as_index=False).agg({
    'TotalImprovement': 'sum',
    'NormalizedGoalsPlusAssistsImprovement': 'sum',
    'NormalizedProgressivePassesImprovement': 'sum',
    'NormalizedProgressiveCarriesImprovement': 'sum'
})

# Sort by total improvement and get the top 20 rising stars
rising_stars = rising_stars.round(2)
rising_stars = rising_stars.sort_values(by='TotalImprovement', ascending=False).head(20)


In [50]:
# Create an interactive bar chart to visualize the top 20 rising stars
fig = px.bar(
    rising_stars,
    x='Player',  
    y='TotalImprovement', 
    color='Player',
    hover_data=[
        'Age',
        'NormalizedGoalsPlusAssistsImprovement',
        'NormalizedProgressivePassesImprovement',
        'NormalizedProgressiveCarriesImprovement'
    ],  # Additional data to show on hover
    title="Top 20 Rising Stars: Total Improvement Across Seasons",
    labels={
        'TotalImprovement': 'Total Normalized Improvement',
        'Player': 'Player Name',
        'Age': 'Age'
    }
)

# Customize the layout
fig.update_layout(
    xaxis={'categoryorder': 'total descending'},  # Order players by total improvement
    showlegend=False  # Hide the legend since each bar is already labeled
)

# Show the plot
fig.show()